In [ ]:
!pip install newspaper3k
!pip install beautifulsoup4
!pip install requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=8221704d1d411d4985f544015c6d10b258349771776592762c264b15267d06a5
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=bdbbc06c238475097ce4a7b14962ff022e45be87d46ef139f185707888e12726
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc

# **GET URL**

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def crawl_url(base_url, total_pages):
    try:
        with open('url_list.csv', mode='w', newline='') as csv_file:
            fieldnames = ['URL']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()

            for page in range(70, total_pages + 1):
                url = f'{base_url}&sortby=time&sorttime=0&page={page}'
                response = requests.get(url)
                response.raise_for_status()
                html_content = response.text
                soup = BeautifulSoup(html_content, 'html.parser')
                list_berita_div = soup.find('div', class_='list media_rows list-berita')
                if list_berita_div:
                    article_links = list_berita_div.find_all('a', href=True)
                    for article_link in article_links:
                        url = article_link['href']
                        if url.startswith('https://news.detik.com/berita/'):
                            writer.writerow({'URL': url})

    except requests.exceptions.RequestException as e:
        print("Error during HTTP request:", e)

base_url = 'https://www.detik.com/search/searchall?query=kriminal&siteid=2'
total_pages = 427 #427
crawl_url(base_url, total_pages)



# **GET DETAIL ARTICLE**

In [ ]:
from newspaper import Article
import pandas as pd
import time
import os
import nltk
nltk.download('punkt')


file_csv = '/content/url_list.csv'
tmp_folder = '/content/tmp'  # Folder sementara untuk file yang sudah di-crawl

# Buat folder sementara jika belum ada
if not os.path.exists(tmp_folder):
    os.makedirs(tmp_folder)

def get_article_info(link):
    article = Article(link, language='id')
    article.download()
    article.parse()
    article.nlp()
    return {
        'Judul': article.title,
        'Penulis': article.authors,
        'Tanggal publikasi': article.publish_date,
        'Teks berita': article.text,
        'Ringkasan' : article.summary,
    }

df = pd.read_csv(file_csv)

articles_data = []

start_time = time.time()  # Waktu mulai

for idx, link in enumerate(df['URL']):
    try:
        article_info = get_article_info(link)
        articles_data.append(article_info)

        # Menyimpan data sementara setiap 100 artikel
        if (idx + 1) % 100 == 0:
            tmp_file = os.path.join(tmp_folder, f'tmp_{idx + 1}.csv')
            df_tmp = pd.DataFrame(articles_data)
            df_tmp.to_csv(tmp_file, index=False)
            print(f"Data untuk {idx + 1} artikel tersimpan sementara di {tmp_file}")

            # Menjeda selama 5 detik sebelum melanjutkan
            time.sleep(5)

    except Exception as e:
        print("Gagal mengambil informasi dari link:", link)
        print("Error:", str(e))

# Simpan data akhir dalam file CSV
df_result = pd.DataFrame(articles_data)
output_csv = '/content/raw_data.csv'
df_result.to_csv(output_csv, index=False)

end_time = time.time()  # Waktu berakhir
running_time = end_time - start_time

# Menghapus folder sementara setelah selesai
for tmp_file in os.listdir(tmp_folder):
    tmp_path = os.path.join(tmp_folder, tmp_file)
    os.remove(tmp_path)
os.rmdir(tmp_folder)


print("Data berhasil disimpan di", output_csv)
print("Waktu yang diperlukan:", running_time, "detik")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Gagal mengambil informasi dari link: https://news.detik.com/berita/d-7160447/tebas-tangan-korban-hingga-putus-anggota-geng-motor-di-cilegon-ditangkap
Error: Article `download()` failed with HTTPSConnectionPool(host='news.detik.com', port=443): Read timed out. (read timeout=7) on URL https://news.detik.com/berita/d-7160447/tebas-tangan-korban-hingga-putus-anggota-geng-motor-di-cilegon-ditangkap
Data untuk 100 artikel tersimpan sementara di /content/tmp/tmp_100.csv
Data untuk 200 artikel tersimpan sementara di /content/tmp/tmp_200.csv
Data untuk 300 artikel tersimpan sementara di /content/tmp/tmp_300.csv
Data untuk 400 artikel tersimpan sementara di /content/tmp/tmp_400.csv
Data untuk 500 artikel tersimpan sementara di /content/tmp/tmp_500.csv
Data untuk 600 artikel tersimpan sementara di /content/tmp/tmp_600.csv
Gagal mengambil informasi dari link: https://news.detik.com/berita/d-6916141/ketua-pengadilan-yang-digerebek-mertua-sedang-selingkuh-di-hotel-dipecat
Error: Article `download()`

In [ ]:
from newspaper import Article
article = Article('https://www.cnnindonesia.com/tv/20221208234440-404-884961/video-waspada-modus-baru-penipuan-online','id') # Indonesia
article.download()
article.parse()
print('Judul:', article.title)
print('Penulis:', article.authors)
print('Tanggal publikasi:', article.publish_date)
print('Teks berita:', article.text)
print('Gambar utama:', article.top_image)